In [165]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

En este script procesare los datos de la msma forma que el script V1 con la diferencia de eliminar los valores menos frecuentes y 
alguna transformacion externa de caracteristica. 
La prediccion la realizare con XGBoost.

In [46]:
df_train = pd.read_csv('train.csv', index_col='id')
df_train

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
id,,,,,,,,,,,,,,,,,
0,Male,24.443011,1.699998,81.669950,yes,yes,2.000000,2.983297,Sometimes,no,2.763573,no,0.000000,0.976473,Sometimes,Public_Transportation,Overweight_Level_II
1,Female,18.000000,1.560000,57.000000,yes,yes,2.000000,3.000000,Frequently,no,2.000000,no,1.000000,1.000000,no,Automobile,Normal_Weight
2,Female,18.000000,1.711460,50.165754,yes,yes,1.880534,1.411685,Sometimes,no,1.910378,no,0.866045,1.673584,no,Public_Transportation,Insufficient_Weight
3,Female,20.952737,1.710730,131.274851,yes,yes,3.000000,3.000000,Sometimes,no,1.674061,no,1.467863,0.780199,Sometimes,Public_Transportation,Obesity_Type_III
4,Male,31.641081,1.914186,93.798055,yes,yes,2.679664,1.971472,Sometimes,no,1.979848,no,1.967973,0.931721,Sometimes,Public_Transportation,Overweight_Level_II
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20753,Male,25.137087,1.766626,114.187096,yes,yes,2.919584,3.000000,Sometimes,no,2.151809,no,1.330519,0.196680,Sometimes,Public_Transportation,Obesity_Type_II
20754,Male,18.000000,1.710000,50.000000,no,yes,3.000000,4.000000,Frequently,no,1.000000,no,2.000000,1.000000,Sometimes,Public_Transportation,Insufficient_Weight
20755,Male,20.101026,1.819557,105.580491,yes,yes,2.407817,3.000000,Sometimes,no,2.000000,no,1.158040,1.198439,no,Public_Transportation,Obesity_Type_II


In [121]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20758 entries, 0 to 20757
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Gender                          20758 non-null  object 
 1   Age                             20758 non-null  float64
 2   Height                          20758 non-null  float64
 3   Weight                          20758 non-null  float64
 4   family_history_with_overweight  20758 non-null  object 
 5   FAVC                            20758 non-null  object 
 6   FCVC                            20758 non-null  float64
 7   NCP                             20758 non-null  float64
 8   CAEC                            20758 non-null  object 
 9   SMOKE                           20758 non-null  object 
 10  CH2O                            20758 non-null  float64
 11  SCC                             20758 non-null  object 
 12  FAF                             20758

In [34]:
# Castear variables float a enteras (teniendo en cuenta las fracciones de algunas caracteristicas)
def feature_float_int (df):
    df_copy = df.copy()
    columns_to_transform = ['Age', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE']

    for i in columns_to_transform:
        if i == 'Age':                         
            df_copy[i]=df_copy[i].astype(int)
        else: df_copy[i]=round(df_copy[i]).astype(int)
        
    if any(df_copy[column].dtype !='int32' for column in columns_to_transform):
        print('Error: No se casteo correctamente alguna caracteristica')
        return df_copy
    
    print('Casteo realizado con exito')
    return df_copy

In [47]:
df_train = feature_float_int(df_train)
df_train

Casteo realizado con exito


,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
id,,,,,,,,,,,,,,,,,
0,Male,24,1.699998,81.669950,yes,yes,2,3,Sometimes,no,3,no,0,1,Sometimes,Public_Transportation,Overweight_Level_II
1,Female,18,1.560000,57.000000,yes,yes,2,3,Frequently,no,2,no,1,1,no,Automobile,Normal_Weight
2,Female,18,1.711460,50.165754,yes,yes,2,1,Sometimes,no,2,no,1,2,no,Public_Transportation,Insufficient_Weight
3,Female,20,1.710730,131.274851,yes,yes,3,3,Sometimes,no,2,no,1,1,Sometimes,Public_Transportation,Obesity_Type_III
4,Male,31,1.914186,93.798055,yes,yes,3,2,Sometimes,no,2,no,2,1,Sometimes,Public_Transportation,Overweight_Level_II
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20753,Male,25,1.766626,114.187096,yes,yes,3,3,Sometimes,no,2,no,1,0,Sometimes,Public_Transportation,Obesity_Type_II
20754,Male,18,1.710000,50.000000,no,yes,3,4,Frequently,no,1,no,2,1,Sometimes,Public_Transportation,Insufficient_Weight
20755,Male,20,1.819557,105.580491,yes,yes,2,3,Sometimes,no,2,no,1,1,no,Public_Transportation,Obesity_Type_II


In [5]:
# Eliminar valores atipicos de Age y Height

def delete_outliers (df):
    df_copy = df.copy()
    out=[]

    print('Tamaño del df antes de remover outliers:',df_copy.shape)
    columns_outliers = ['Age', 'Height']

    for i in columns_outliers:
        q1 = df_copy[i].quantile(0.25)
        q3 = df_copy[i].quantile(0.75)
        iqr = q3-q1
        Lower_tail = q1 - 1.5 * iqr
        Upper_tail = q3 + 2.0 * iqr
    
        outliers = df_copy[(df_copy[i] < Lower_tail) | (df_copy[i] > Upper_tail)]
        out.extend(outliers.index)

    out = list(set(out))
    df_copy = df_copy.drop(out)
    print("Outliers removidos. Tamaño del nuevo df:", df_copy.shape)
    return df_copy

In [48]:
df_train= delete_outliers (df_train)

Tamaño del df antes de remover outliers: (20758, 17)
Outliers removidos. Tamaño del nuevo df: (20251, 17)


In [128]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20251 entries, 0 to 20757
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Gender                          20251 non-null  object 
 1   Age                             20251 non-null  int32  
 2   Height                          20251 non-null  float64
 3   Weight                          20251 non-null  float64
 4   family_history_with_overweight  20251 non-null  object 
 5   FAVC                            20251 non-null  object 
 6   FCVC                            20251 non-null  int32  
 7   NCP                             20251 non-null  int32  
 8   CAEC                            20251 non-null  object 
 9   SMOKE                           20251 non-null  object 
 10  CH2O                            20251 non-null  int32  
 11  SCC                             20251 non-null  object 
 12  FAF                             20251

In [44]:
# Funcion que redondea a dos decimales altura y peso
def round_weight_height(df):
    df_copy = df.copy()
    columns = ['Height', 'Weight']

    for i in columns:
        df_copy[i] = round(df_copy[i],2)
    
    return df_copy

In [49]:
df_train = round_weight_height(df_train)

In [45]:
# Funcion que calcula el IMC y la añade como variable nueva
def imc_calculator(df):
    df_copy = df.copy()
    
    df_copy['IMC'] = round(df_copy['Weight']/(df_copy['Height']**2),1)
    return df_copy
    

In [50]:
df_train=imc_calculator(df_train)

In [51]:
# Movemos la variable objetivo al final de df
column_pop = df_train.pop('NObeyesdad')
df_train.insert(17, 'NObeyesdad', column_pop)

In [54]:
df_train.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,IMC,NObeyesdad
id,,,,,,,,,,,,,,,,,,
0,Male,24,1.70,81.67,yes,yes,2,3,Sometimes,no,3,no,0,1,Sometimes,Public_Transportation,28.3,Overweight_Level_II
1,Female,18,1.56,57.00,yes,yes,2,3,Frequently,no,2,no,1,1,no,Automobile,23.4,Normal_Weight
2,Female,18,1.71,50.17,yes,yes,2,1,Sometimes,no,2,no,1,2,no,Public_Transportation,17.2,Insufficient_Weight
3,Female,20,1.71,131.27,yes,yes,3,3,Sometimes,no,2,no,1,1,Sometimes,Public_Transportation,44.9,Obesity_Type_III
4,Male,31,1.91,93.80,yes,yes,3,2,Sometimes,no,2,no,2,1,Sometimes,Public_Transportation,25.7,Overweight_Level_II


In [74]:
# Separamos en X e y
X = df_train.iloc[:, :-1]
y = df_train.iloc[:, 17]

In [80]:
# Funcion para eliminar columnas cuyas distribucion es < 90%
def del_column(X):
    X_copy = X.copy()

    X_copy.drop(['SMOKE', 'SCC'], inplace=True, axis=1)
    return X_copy

In [81]:
X = del_column(X)

In [83]:
# Codificar valores categoricos
def dummies_variable (X):
    X_copy = X.copy()
    categorical_var = X_copy.select_dtypes(include='object')

    # Creamos instancia de encoder
    encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    encoder.fit(categorical_var)
    # Codificamos
    categorical_encoded = encoder.transform(categorical_var)

    # Convertimos a df
    X_copy = pd.DataFrame(categorical_encoded, columns=encoder.get_feature_names_out(categorical_var.columns))

    return X_copy

In [84]:
X_categorical = dummies_variable (X)

In [85]:
X_categorical.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20251 entries, 0 to 20250
Data columns (total 18 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Gender_Female                       20251 non-null  float64
 1   Gender_Male                         20251 non-null  float64
 2   family_history_with_overweight_no   20251 non-null  float64
 3   family_history_with_overweight_yes  20251 non-null  float64
 4   FAVC_no                             20251 non-null  float64
 5   FAVC_yes                            20251 non-null  float64
 6   CAEC_Always                         20251 non-null  float64
 7   CAEC_Frequently                     20251 non-null  float64
 8   CAEC_Sometimes                      20251 non-null  float64
 9   CAEC_no                             20251 non-null  float64
 10  CALC_Frequently                     20251 non-null  float64
 11  CALC_Sometimes                      20251

In [86]:
# Normalizar los datos nuemericos
def normalizer_numerical (X):
    X_copy = X.copy()
    numerical_val = X_copy.select_dtypes(include='number')
    columns_name = numerical_val.columns
    # escalado y transformacion
    scaler = StandardScaler()
    nuemrical_scaled = scaler.fit_transform(numerical_val)

    # Convertimos a df
    numerical_df = pd.DataFrame(nuemrical_scaled, columns=columns_name)

    return numerical_df

In [87]:
X_numerical = normalizer_numerical (X)

In [88]:
X_numerical.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20251 entries, 0 to 20250
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Age     20251 non-null  float64
 1   Height  20251 non-null  float64
 2   Weight  20251 non-null  float64
 3   FCVC    20251 non-null  float64
 4   NCP     20251 non-null  float64
 5   CH2O    20251 non-null  float64
 6   FAF     20251 non-null  float64
 7   TUE     20251 non-null  float64
 8   IMC     20251 non-null  float64
dtypes: float64(9)
memory usage: 1.4 MB


In [89]:
# Funcion para concatenar categoricas y numericas
def concat_types(num,cat):
    X_ = pd.concat([num,cat], axis=1)
    return X_

In [90]:
X = concat_types(X_categorical,X_numerical)

In [98]:
# Funcion global de procesamiento de datos (en conjuntos de prueba)
def processing_data(df):
    df = feature_float_int(df)
    df=round_weight_height(df)
    df=imc_calculator(df)
    df = del_column(df)
    df_dummies = dummies_variable (df)
    df_normalizer=normalizer_numerical (df)
    df_concat = concat_types(df_dummies,df_normalizer)

    return df_concat

In [93]:
# Entrenar modelo y mostrar resultados entrenamiento y validacion
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=123, stratify=y)

In [99]:
X_train.shape

(16200, 27)

In [100]:
X_val.shape

(4051, 27)

VAMOS A REALIZAR VARIAS PREDICCIONES CON 3 ALGORITMOS

In [102]:
# Creamos un diccionario para establecer los labels en la variable y (necesario en el algoritmo XGBoost)
label_mapping = {
    'Insufficient_Weight': 0,
    'Normal_Weight': 1,
    'Obesity_Type_I': 2,
    'Obesity_Type_II': 3,
    'Obesity_Type_III': 4,
    'Overweight_Level_I': 5,
    'Overweight_Level_II': 6
}

In [103]:
# Aplicar el mapeo a las etiquetas de entrenamiento
y_train_encoded = y_train.map(label_mapping)

# Aplicar el mapeo a las etiquetas de validacion
y_val_encoded = y_val.map(label_mapping)

In [161]:
# Algoritmo XGBoost
model = XGBClassifier(n_estimators=200, max_depth=2, learning_rate=0.5, objective='multi:softmax')

In [162]:
model.fit(X_train, y_train_encoded)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.5, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, objective='multi:softmax', ...)

In [163]:
y_pred = model.predict(X_val)

In [171]:
# Validacion cruzada
scores = cross_val_score(model, X_train, y_train_encoded, cv=10) 
print("Precisión media de la validación cruzada:", "{:.2f}".format(scores.mean()))

Precisión media de la validación cruzada: 0.90


La validacion media es del 90% de precision

In [172]:
# Funcion que da el valor categorico a cada clave del diccionario labels (necesario para volver a tener los valores como texto)
def label_to_text (pred, label_mapping):
    pred_copy = pred

    # Obtener las etiquetas originales en el mismo orden que los valores numéricos
    original_labels = [key for key, value in sorted(label_mapping.items(), key=lambda x: x[1])]

    # Mapear los valores numéricos a las etiquetas originales
    text_predictions = [original_labels[value] for value in pred_copy]
    return text_predictions

Predecir con el conjunto de prueba y el algoritmo mas alto (XGBoost)

In [174]:
df_prueba = pd.read_csv('test.csv')
df_prueba

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,20758,Male,26.899886,1.848294,120.644178,yes,yes,2.938616,3.000000,Sometimes,no,2.825629,no,0.855400,0.000000,Sometimes,Public_Transportation
1,20759,Female,21.000000,1.600000,66.000000,yes,yes,2.000000,1.000000,Sometimes,no,3.000000,no,1.000000,0.000000,Sometimes,Public_Transportation
2,20760,Female,26.000000,1.643355,111.600553,yes,yes,3.000000,3.000000,Sometimes,no,2.621877,no,0.000000,0.250502,Sometimes,Public_Transportation
3,20761,Male,20.979254,1.553127,103.669116,yes,yes,2.000000,2.977909,Sometimes,no,2.786417,no,0.094851,0.000000,Sometimes,Public_Transportation
4,20762,Female,26.000000,1.627396,104.835346,yes,yes,3.000000,3.000000,Sometimes,no,2.653531,no,0.000000,0.741069,Sometimes,Public_Transportation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13835,34593,Male,23.327836,1.721384,78.030383,yes,no,2.813234,3.000000,Sometimes,no,1.000000,no,0.807076,0.778632,Sometimes,Public_Transportation
13836,34594,Female,29.000000,1.590000,62.000000,no,yes,3.000000,3.000000,Sometimes,no,2.000000,no,0.000000,0.000000,Sometimes,Public_Transportation
13837,34595,Female,22.935612,1.585547,44.376637,no,yes,3.000000,2.273740,Frequently,no,2.000000,no,1.949840,1.000000,Sometimes,Public_Transportation
13838,34596,Male,21.000000,1.620000,53.000000,yes,yes,2.000000,3.000000,Sometimes,no,2.000000,no,3.000000,2.000000,no,Public_Transportation


In [175]:
# Guardamos id
id_ = df_prueba['id']

In [176]:
# Eliminamos id
df_prueba.drop('id', axis=1, inplace=True)

In [177]:
df_prueba = processing_data(df_prueba)

Casteo realizado con exito


In [180]:
# Eliminamos la columna 'CALC_Always' debido a que no esta entrenada en el conjunto de entrenamiento.
df_prueba.drop('CALC_Always', axis=1, inplace=True)

In [182]:
df_prueba.head()

,Gender_Female,Gender_Male,family_history_with_overweight_no,family_history_with_overweight_yes,FAVC_no,FAVC_yes,CAEC_Always,CAEC_Frequently,CAEC_Sometimes,CAEC_no,...,MTRANS_Walking,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE,IMC
0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.402742,1.699294,1.273625,0.981014,0.339134,1.377761,0.025863,-0.935509,0.611880
1,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,-0.468051,-1.109309,-0.818989,-0.794948,-2.416146,1.377761,0.025863,-0.935509,-0.522949
2,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.402742,-0.659932,0.927409,0.981014,0.339134,1.377761,-1.125073,-0.935509,1.372457
3,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,-0.642210,-1.671029,0.623704,-0.794948,0.339134,1.377761,-1.125073,-0.935509,1.577692
4,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.402742,-0.772276,0.668513,0.981014,0.339134,1.377761,-1.125073,0.579166,1.131004


In [183]:
# Prediccion del modelo entrenado en el de prueba
prediction_ = model.predict(df_prueba)

In [184]:
# Convertimos a texto cada valor, dado el diccionario creado anteriormente
prediction_text = label_to_text(prediction_, label_mapping)

In [185]:
# Creamos el dataset de envio con la estructura exigida
submission = {'id': id_, 'NObeyesdad':prediction_text}

In [186]:
submit_v2 = pd.DataFrame(submission)
submit_v2

,id,NObeyesdad
0,20758,Obesity_Type_II
1,20759,Overweight_Level_I
2,20760,Obesity_Type_III
3,20761,Obesity_Type_II
4,20762,Obesity_Type_III
...,...,...
13835,34593,Overweight_Level_II
13836,34594,Normal_Weight
13837,34595,Insufficient_Weight
13838,34596,Normal_Weight


In [187]:
#Guardamos
submit_v2.to_csv('submitV2.csv', index=False)